In [1]:
import numpy as np
import pandas as pd
import os
import re
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


In [2]:
#import wfdb
def split_data(features_df):
    cols = features_df.columns
    #cols_remove = [['session','participant','index','target']]
    cols_remove = ['session','participant','trial','gesture','filename']
    target = ['gesture']
    cols_keep = [x for x in cols if x not in cols_remove]
    #print(cols_keep)
    X = np.array(features_df[cols_keep])
    Y = np.array(features_df[target]).ravel()
    X_train,X_test,Y_train,Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
    return [X_train,X_test,Y_train,Y_test]


In [4]:
features_df = pd.read_csv('/Users/smoothoperator/Documents/GitHub/DS5500-project/models/svm/features_v2.csv')
features_df.head()
features_df.describe()
### Total of 18 gestures to be predicted.

,session,trial,crest_factor_F1,dasd_F1,diffvar_F1,form_factor_F1,iemg_F1,kurtosis_F1,kurtosis_f_F1,mav_F1,...,rms_W9,skew_W9,skew_f_W9,ssi_W9,sum_f_W9,var_f_W9,wflen_W9,willison_W9,gesture,participant
count,15351.000000,15351.000000,15351.000000,15351.000000,15351.000000,1.535100e+04,15351.000000,15351.000000,15351.000000,15351.000000,...,15351.000000,15351.000000,15351.000000,15351.000000,15351.000000,1.535100e+04,15351.000000,15351.000000,15351.000000,15351.000000
mean,2.000000,4.000000,6.047087,0.019260,0.000694,1.834416e+14,378.941714,2.177910,74.926579,0.037006,...,0.033153,-0.320471,4.974771,17.958520,0.119607,1.427294e-09,86.791692,12.251710,9.000000,22.000000
std,0.816523,2.000065,1.710467,0.013658,0.001258,2.272825e+16,269.007735,8.331980,185.944373,0.026270,...,0.025587,0.526493,2.468868,53.344938,0.180624,1.110527e-08,60.281965,38.878098,4.899139,12.410078
min,1.000000,1.000000,3.362290,0.001450,0.000003,-8.242829e+06,24.291688,-0.064162,8.819885,0.002372,...,0.002167,-6.439284,1.203108,0.048079,0.000261,9.661786e-16,12.338405,0.000000,1.000000,1.000000
25%,1.000000,2.000000,5.032689,0.010531,0.000135,-1.930748e+03,217.208917,0.784624,27.823932,0.021212,...,0.015940,-0.484321,3.492647,2.601885,0.035006,2.239063e-11,46.187158,0.000000,5.000000,11.000000
50%,2.000000,4.000000,5.684269,0.015637,0.000299,-4.788446e+02,314.682796,1.293862,42.506134,0.030731,...,0.027160,-0.324770,4.360644,7.553905,0.069141,9.472647e-11,71.530419,3.000000,9.000000,22.000000
75%,3.000000,6.000000,6.596594,0.023628,0.000698,1.835671e+03,459.517967,2.232629,70.617643,0.044875,...,0.043588,-0.168981,5.650867,19.454926,0.129741,3.754907e-10,110.964348,12.000000,13.000000,33.000000
max,3.000000,7.000000,42.678651,0.136473,0.021250,2.816011e+18,3675.686287,580.944410,5768.991217,0.358954,...,0.712933,34.144931,45.090940,5204.725180,3.006518,9.475449e-07,636.512868,974.000000,17.000000,43.000000



**Method 1:** SVC

**Method 2:** Semi-Supervised Clustering using DBScan + RFC or KNN

In [5]:
cols = features_df.columns
#cols_remove = [['session','participant','index','target']]
cols_remove = ['session','participant','trial','gesture','filename']
target = ['gesture']
cols_keep = [x for x in cols if x not in cols_remove]
X = np.array(features_df[cols_keep])
Y = np.array(features_df[target]).ravel()
X_train,X_test,Y_train,Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)


### Method 1
#### Kernel RBF

In [13]:

clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(X_train, Y_train)
Y_labels_svc_1 = clf.predict(X_test)
print(classification_report(Y_test, Y_labels_svc_1))

              precision    recall  f1-score   support

           1       0.69      0.65      0.67       314
           2       0.54      0.71      0.61       300
           3       0.71      0.67      0.69       312
           4       0.61      0.65      0.63       303
           5       0.84      0.75      0.79       298
           6       0.93      0.80      0.86       300
           7       0.69      0.69      0.69       305
           8       0.76      0.85      0.80       279
           9       0.70      0.76      0.73       268
          10       0.87      0.77      0.82       313
          11       0.97      0.94      0.96       286
          12       0.93      0.90      0.91       296
          13       0.94      0.90      0.92       300
          14       0.91      0.89      0.90       272
          15       0.85      0.90      0.87       320
          16       0.75      0.62      0.68       315
          17       0.80      0.97      0.88       285

    accuracy              


#### Kernel Poly

In [7]:
clf = make_pipeline(StandardScaler(), SVC(gamma='auto',kernel = 'poly'))
clf.fit(X_train, Y_train)
Y_labels_svc_2 = clf.predict(X_test)
print(classification_report(Y_test, Y_labels_svc_2))

              precision    recall  f1-score   support

           1       0.64      0.34      0.45       314
           2       0.10      0.96      0.19       300
           3       0.74      0.06      0.12       312
           4       0.37      0.24      0.29       303
           5       0.73      0.25      0.37       298
           6       0.65      0.18      0.29       300
           7       0.69      0.08      0.15       305
           8       0.46      0.39      0.42       279
           9       0.64      0.37      0.47       268
          10       0.80      0.18      0.29       313
          11       0.95      0.82      0.88       286
          12       0.87      0.63      0.73       296
          13       0.94      0.47      0.62       300
          14       0.91      0.43      0.58       272
          15       0.87      0.26      0.40       320
          16       0.81      0.23      0.35       315
          17       0.94      0.96      0.95       285

    accuracy              

#### Kernel Sigmoid

In [8]:
clf = make_pipeline(StandardScaler(), SVC(gamma='auto',kernel = 'sigmoid'))
clf.fit(X_train, Y_train)
Y_labels_svc_3 = clf.predict(X_test)
print(classification_report(Y_test, Y_labels_svc_3))

              precision    recall  f1-score   support

           1       0.45      0.46      0.45       314
           2       0.34      0.54      0.41       300
           3       0.65      0.46      0.54       312
           4       0.51      0.44      0.47       303
           5       0.38      0.44      0.41       298
           6       0.80      0.52      0.63       300
           7       0.59      0.50      0.55       305
           8       0.79      0.54      0.64       279
           9       0.51      0.56      0.53       268
          10       0.69      0.35      0.47       313
          11       0.73      0.73      0.73       286
          12       0.85      0.69      0.76       296
          13       0.89      0.67      0.76       300
          14       0.70      0.66      0.68       272
          15       0.68      0.56      0.61       320
          16       0.30      0.52      0.38       315
          17       0.49      0.87      0.63       285

    accuracy              

### RBF Kernel gives the best accuracy and recall score of .79 each.


## Method 2

Now we test the semi supervised approach by using a DBScan, to cluster similar groups 


In [9]:
#Get Sample of data for clustering (2/3 for clustering)
features_clustering = features_df.sample(frac=0.67)
#features_clustering.gesture.value_counts()
#we get roughly the same number of sampels from each gesture

#Get train test splits
[X_clustering, X_rf, Y_clustering, Y_rf] = split_data(features_clustering)

#Perform DB Scan and get the most popular gesture as the Cluster Gesture
scaler = StandardScaler()
X_clustering_scaled = scaler.fit_transform(X_clustering)

clustering = DBSCAN(eps=2, min_samples=5).fit(X_clustering_scaled)
clustering.labels_

array([-1, -1, -1, ..., -1, -1, -1])

### Method 2 with PCA

In [10]:
pca = PCA(n_components=10)
#Get Sample of data for clustering (2/3 for clustering)
features_clustering = features_df.sample(frac=0.67)
#features_clustering.gesture.value_counts()
#we get roughly the same number of sampels from each gesture

#Get train test splits
[X_clustering, X_rf, Y_clustering, Y_rf] = split_data(features_clustering)

#Perform DB Scan and get the most popular gesture as the Cluster Gesture
scaler = StandardScaler()
X_clustering_scaled= scaler.fit_transform(X_clustering)
X_rf_scaled = scaler.transform(X_rf)

X_clustering_reduced = pca.fit_transform(X_clustering_scaled)
X_rf_reduced = pca.transform(X_rf_scaled)
print(pca.explained_variance_ratio_.sum())

0.6888949884223973


In [11]:
print(X_clustering.shape)
print(X_clustering_reduced.shape)

(6890, 756)
(6890, 10)


In [12]:
clustering = DBSCAN(eps=2,metric='euclidean', min_samples=5).fit(X_clustering_reduced)
np.unique(clustering.labels_)

array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13])

## Results 
